原来的代码能够保证秘密共享的数据没有被修改，享受数字签名带来诸多好处。

但是原来的秘密共享方案并不是完全安全的，因为之前的假设是秘密共享的参与方都是诚实的

实际上，如果参与方中有人不诚实，其他参与方虽然能够通过验证签名来得知图像（不是原来的图像）被修改过，但却不能得知谁传输了假的子秘密

为了解决参与方不诚实的问题，需要对原本的门限方案进行改进。

        注意：

        1. 为了计算方便，参与方的序号统一从1开始

        2. 需要实现的是基于Shamir秘密共享构造的可验证秘密共享方案

        3. 为了方便验证和处理，将彩色通道分成了R、G、B三通道，并单独验证其正确性

首先，实现简单的可证门限秘密共享，其中共享的是数字，成功后将数字换成R、G、B三通道灰度图的像素值即可。

In [17]:
import random
import sympy

# 秘密数secret、份额数num_share、恢复秘密最小需要threshold
secret = 3
num_shares = 6
threshold = 4

# 生成大素数和生成元
'''
这段代码的作用是生成用于实现Feldman门限方案的大素数和生成元。

sympy.randprime(2**12, 2**16) 生成一个大素数:sympy.randprime 是 SymPy 库中的一个函数，用于生成指定范围内的随机素数。

这里生成一个范围从 2 的 12 次方到 2 的 16 次方之间的随机素数,并将其赋给 p 变量。

random.randint(2, p - 2) 生成生成元：生成元是一个整数，它被选择为一个与素数 p 相关联的整数。

这里通过随机选择一个范围在 2 到 p - 2 之间的整数,生成一个生成元,并将其赋给 g 变量。
'''

# p = sympy.randprime(2**12, 2**16)
# g = random.randint(2, p - 2)
p = 11
g = 3
q = 5


# 这个函数是用来生成随机多项式的系数的，当然，在后续的计算中，每张图片中的所有像素值都是用同样的随机多项式，好方便分发验证消息
def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(1, 10)) # 生成随机系数，添加到列表中
    return coefficients

coefficients = generate_coefficients(secret, threshold) # 这里调用上述函数生成随机多项式，用于后续的秘密共享
print(coefficients)

# Shamir的秘密共享方案 - 秘密分割
def generate_shares(num_shares):
    shares = []
    for i in range(1, num_shares + 1):
        share = 0
        for power, c  in enumerate(coefficients):
            share += c * (i ** power) % q
        shares.append((i, share))
    return shares

# Feldman门限方案 - 生成验证信息：接收两个参数p和g
def generate_verification_information(p, g):
    # verification_info用来存放承诺
    verification_info = []
    # 这个循环用来生成各方承诺
    for power, c in enumerate(coefficients):
        promise = pow(g, c) % p
        verification_info.append(promise)
    return verification_info

# Feldman门限方案 - 验证份额：接收子秘密share，接收承诺，接收素数p，接收生成元g，接收参与方序号i
def verify_share(share, verification_info, p, g, i):
    wait_verification = pow(g, share) % p
    print(wait_verification)
    verification = 1
    for power, promise in enumerate(verification_info):
        i_power = pow(i, power)
        verification *= pow(promise, i_power) % p
    verification %= p
    print(verification)
    if wait_verification == verification % p:
        return True
    return False

# 验证所有份额
def verify_shares(shares, verification_info, p, g):
    for i, share in shares:
        # 对于每个子秘密，都验证一次
        print(i)
        if not verify_share(share, verification_info, p, g, i):
            return False
    return True

# 分割秘密
shares = generate_shares(num_shares)
print(shares)
# 生成验证信息
verification_info = generate_verification_information(p, g)
print(verification_info)
# 模拟验证

print(p)
print(g)


is_valid = verify_shares(shares, verification_info, p, g)

if is_valid:
    print("所有份额有效")
else:
    print("至少有一个份额无效")

[3, 3, 8, 9]
[(1, 13), (2, 8), (3, 12), (4, 9), (5, 3), (6, 13)]
[5, 5, 5, 4]
11
3
1
5
5
2
5
5
3
9
9
4
4
4
5
5
5
6
5
5
所有份额有效


上述方案有一个限制，及 p 和 q 都得是大素数，且 q 是 p - 1 的因子，由于图片的恢复需要模256，显然不符合上述条件，所以上述方案并不能实现图像的可证秘密共享，下面尝试在原来方案的基础上，利用两次数字签名保证来实现图像的可证秘密共享

1. 产生RSA公私钥对，并以文件的形式保存，使用Python的Crypto库。其中公钥是公开的，私钥是秘密分发者个人持有，其他人无法得到（密钥生成后打印完成信息）

In [18]:
from Crypto.PublicKey import RSA

# 生成RSA密钥对
key = RSA.generate(2048)  # 这里使用2048位的密钥大小，您可以根据需要选择不同的密钥大小

# 获取私钥
private_key = key.export_key()

# 获取公钥
public_key = key.publickey().export_key()

# 保存私钥到文件
with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/private_key.pem", "wb") as private_key_file:
    private_key_file.write(private_key)

# 保存公钥到文件
with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/public_key.pem", "wb") as public_key_file:
    public_key_file.write(public_key)

# 完成生成
print("RSA公私钥对文件已成功生成")

RSA公私钥对文件已成功生成


2. 对原始图像进行数字签名（签名完成后打印完成信息）

In [19]:
from PIL import Image
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

# 提取图像数据的像素数据
image = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png")
pixel_data = list(image.getdata())

# 计算图像数据的SHA-256哈希值
sha256_hash = SHA256.new()
for pixel in pixel_data:
    sha256_hash.update(bytes(pixel))

# 从密钥文件加载私钥
with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/private_key.pem", "rb") as private_key_file:
    private_key_data = private_key_file.read()

# 生成私钥对象
private_key = RSA.import_key(private_key_data)

# 使用私钥对SHA-256哈希值进行签名
signature_bytes = pkcs1_15.new(private_key).sign(sha256_hash)

# 保存数字签名
with open("/Users/colofuluangang/Desktop/code/Shamir/签名文件/signature.sig", "wb") as sig_file:
    sig_file.write(signature_bytes)

# 签名完成
print("原始图像数字签名完成")

原始图像数字签名完成


3. 利用Python的图像处理库来将RGB图像分成单独的R、G、B通道。

In [20]:
from PIL import Image 

def split_channels(image):
    r, g, b = image.split() #使用 split()函数拆分图像为R、G、B通道
    return r, g, b

# 读取图像文件
file_path = "/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png"
image = Image.open(file_path)
width, height = image.size

if image.mode == "RGB":
    print("图像为 RGB 模式")
    r_channel, g_channel, b_channel = split_channels(image)
    # 显示每个通道的图像
    r_channel.save("/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/R通道.png")
    g_channel.save("/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/G通道.png")
    b_channel.save("/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/B通道.png")
    print("图像已分割")
else:
    print("图像不是 RGB 模式")

图像为 RGB 模式
图像已分割


4. 将图片分割成RGB三通道的灰度图像之后，获取每张图像的像素值，然后每张图像利用shamir门限方案产生num_shares个子秘密图像

In [21]:
from random import randint
import random
from PIL import Image
import numpy as np

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
file_path = "/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/R通道.png"
image = Image.open(file_path)
pixels = image.load()

def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(20, 300)) # 生成随机系数，添加到列表中
    return coefficients

# Shamir的秘密共享方案 - 秘密分割
def generate_shares(secret, num_shares, threshold, x, y):
    coefficients = generate_coefficients(secret, threshold)
    for i in range(1, num_shares + 1):
        share = sum(c * (i ** power) for power, c in enumerate(coefficients)) % 256
        images[i - 1].putpixel((x, y), share)

images = []
for i in range(num_shares):
    new_image = Image.new("L", (width, height))
    images.append(new_image)

for y in range(height):
    for x in range(width):
        generate_shares(pixels[x, y], num_shares, threshold, x, y)
        
for idx, img in enumerate(images):
    img.save(f"/Users/colofuluangang/Desktop/code/Shamir/R通道子秘密图片/R{idx}.png")
print(f"生成了{num_shares}个R通道子秘密灰度图像")

生成了6个R通道子秘密灰度图像


In [22]:
from random import randint
import random
from PIL import Image
import numpy as np

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
file_path = "/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/G通道.png"
image = Image.open(file_path)
pixels = image.load()

def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(20, 300)) # 生成随机系数，添加到列表中
    return coefficients


# Shamir的秘密共享方案 - 秘密分割
def generate_shares(secret, num_shares, threshold, x, y):
    coefficients = generate_coefficients(secret, threshold)
    for i in range(1, num_shares + 1):
        share = sum(c * (i ** power) for power, c in enumerate(coefficients)) % 256
        images[i - 1].putpixel((x, y), share)

images = []
for i in range(num_shares):
    new_image = Image.new("L", (width, height))
    images.append(new_image)

for y in range(height):
    for x in range(width):
        generate_shares(pixels[x, y], num_shares, threshold, x, y)
        
for idx, img in enumerate(images):
    img.save(f"/Users/colofuluangang/Desktop/code/Shamir/G通道子秘密图片/G{idx}.png")
print(f"生成了{num_shares}个G通道子秘密灰度图像")

生成了6个G通道子秘密灰度图像


In [23]:
from random import randint
import random
from PIL import Image
import numpy as np

num_shares = 6  # 要创建的份额总数
threshold = 4  # 重构秘密所需的最小份额数量
file_path = "/Users/colofuluangang/Desktop/code/Shamir/RGB通道图片/B通道.png"
image = Image.open(file_path)
pixels = image.load()

def generate_coefficients(secret, threshold):
    coefficients = [secret] # 将秘密值作为系数列表的第一个元素
    for _ in range(threshold - 1):
        coefficients.append(random.randint(20, 300)) # 生成随机系数，添加到列表中
    return coefficients

# Shamir的秘密共享方案 - 秘密分割
images = []
def generate_shares(secret, num_shares, threshold, x, y):
    coefficients = generate_coefficients(secret, threshold)
    for i in range(1, num_shares + 1):
        share = sum(c * (i ** power) for power, c in enumerate(coefficients)) % 256
        images[i - 1].putpixel((x, y), share)


for i in range(num_shares):
    new_image = Image.new("L", (width, height))
    images.append(new_image)

for y in range(height):
    for x in range(width):
        generate_shares(pixels[x, y], num_shares, threshold, x, y)
        
for idx, img in enumerate(images):
    img.save(f"/Users/colofuluangang/Desktop/code/Shamir/B通道子秘密图片/B{idx}.png")
print(f"生成了{num_shares}个B通道子秘密灰度图像")


生成了6个B通道子秘密灰度图像


5. 将同序号的秘密图像的三通道灰度图合成一张彩色图

In [24]:
from PIL import Image

num_shares = 6  # 要创建的份额总数

for i in range(num_shares):
    # 打开三张通道的图像
    r_channel = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/R通道子秘密图片/R{i}.png")
    g_channel = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/G通道子秘密图片/G{i}.png")
    b_channel = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/B通道子秘密图片/B{i}.png")

    # 合并图像
    merged_image = Image.merge("RGB", (r_channel, g_channel, b_channel))

    # 保存合并后的彩色图像
    merged_image.save(f"/Users/colofuluangang/Desktop/code/Shamir/子秘密图像/子秘密{i}.png")

6. 发送方为了保证参与者之间能够验证对方的子秘密图像的正确性，对每个子秘密图像进行数字签名，并设立一个用于检验参与方是否诚实的平台，这个平台能够接收参与方的子秘密图像，然后验证这些参与方是否诚实，在验证成功前参与方之间不会知道其他人的子秘密图像信息，如果验证失败不进行之后的步骤

In [25]:
from PIL import Image
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

# 提取图像数据的像素数据
for i in range(num_shares):
    image = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/子秘密图像/子秘密{i}.png")
    pixel_data = list(image.getdata())

    # 计算图像数据的SHA-256哈希值
    sha256_hash = SHA256.new()
    for pixel in pixel_data:
        sha256_hash.update(bytes(pixel))

    # 从密钥文件加载私钥
    with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/private_key.pem", "rb") as private_key_file:
        private_key_data = private_key_file.read()

    # 生成私钥对象
    private_key = RSA.import_key(private_key_data)

    # 使用私钥对SHA-256哈希值进行签名
    signature_bytes = pkcs1_15.new(private_key).sign(sha256_hash)

    # 保存数字签名
    with open(f"/Users/colofuluangang/Desktop/code/Shamir/签名文件/signature{i}.sig", "wb") as sig_file:
        sig_file.write(signature_bytes)

# 签名完成
print("子秘密图像数字签名完成")

子秘密图像数字签名完成


7. 上述子秘密图像会发给秘密共享参与方，原图像签名文件和各子秘密图像签名文件以及RSA公钥文件都会公开发布，参与方到场后，将自己的子秘密图像交给下述平台，平台可以利用参与方公开的签名文件验证各个参与方子秘密图像的正确性，这里假设到场4位参与方序号为1、2、3、4，首先需要验证各个参与方的图像没有经过修改。如果验证失败，则中断后续的图像恢复操作。

In [26]:
from PIL import Image
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

# 初始化一个标志变量，用于标记是否有验证失败
verification_failed = False

# 打开数字签名文件
for i in range(threshold):
    with open(f"/Users/colofuluangang/Desktop/code/Shamir/签名文件/signature{i}.sig", "rb") as signature_file:
        signature = signature_file.read()

    # 提取图像数据的像素数据
    image = Image.open(f"/Users/colofuluangang/Desktop/code/Shamir/子秘密图像/子秘密{i}.png")
    pixel_data = list(image.getdata())

    # 计算图像数据的SHA-256哈希值
    sha256_hash = SHA256.new()
    for pixel in pixel_data:
        sha256_hash.update(bytes(pixel))

    # 打开公钥文件
    with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/public_key.pem", "rb") as public_key_file:
        public_key_data = public_key_file.read()

    # 创建RSA公钥对象
    public_key = RSA.import_key(public_key_data)

    try:
        # 使用公钥验证签名
        pkcs1_15.new(public_key).verify(sha256_hash, signature)
        print("数字签名验证成功，签名有效。")
    except (ValueError, TypeError):
        print("数字签名验证失败，签名无效。")
        verification_failed = True

# 检查标志，如果有任何验证失败，就退出整个循环
if verification_failed:
    print("至少有一个数字签名验证失败，无法进行后续步骤")


数字签名验证成功，签名有效。
数字签名验证成功，签名有效。
数字签名验证成功，签名有效。
数字签名验证成功，签名有效。


8. 验证成功后，可以继续进行图像恢复，需要将三个子秘密图像分成R、G、B三通道的图像后各自保存，为了测试方便，这里直接用之前的分割的各通道秘密图像，序号为1、2、3、4，实际运行时，序号可以改变。

        注意：下述的代码是重复的，为了方便理解，写了三遍，实际上可以整合在一起，成为一个代码块。

In [27]:
from PIL import Image

if verification_failed == False:
    new_image = Image.new("L", (width, height))
    num_shares = 6  # 要创建的份额总数
    threshold = 4  # 重构秘密所需的最小份额数量

    # 准备用于存储所有子图片数据的列表
    images = []
    for i in range(threshold):
        file_path = f"/Users/colofuluangang/Desktop/code/Shamir/R通道子秘密图片/R{i}.png"
        img = Image.open(file_path)
        images.append(img.load())  # 将像素数据加载到内存，避免多次读取文件

    # Shamir的秘密共享方案 - 优化的秘密重构函数
    def reconstruct_secret(images, x, y):
        secret = 0
        for i in range(1, threshold + 1):
            share = images[i - 1][x, y]
            num, den = 1, 1
            for j in range(1, threshold + 1):
                if i != j:
                    num *= -j
                    den *= (i - j)
            lagrange = share * num / den
            secret += lagrange
        secret %= 256
        return int(secret)

    # 逐位置读取每张图片的灰度值并进行处理
    for y in range(height):
        for x in range(width):
            new_pixels = reconstruct_secret(images, x, y)
            new_image.putpixel((x, y), new_pixels)
            
    new_image.save("/Users/colofuluangang/Desktop/code/Shamir/R通道恢复秘密图片/R恢复.png")

In [28]:
from PIL import Image

if verification_failed == False:
    new_image = Image.new("L", (width, height))
    num_shares = 6  # 要创建的份额总数
    threshold = 4  # 重构秘密所需的最小份额数量

    # 准备用于存储所有子图片数据的列表
    images = []
    for i in range(threshold):
        file_path = f"/Users/colofuluangang/Desktop/code/Shamir/G通道子秘密图片/G{i}.png"
        img = Image.open(file_path)
        images.append(img.load())  # 将像素数据加载到内存，避免多次读取文件

    # Shamir的秘密共享方案 - 优化的秘密重构函数
    def reconstruct_secret(images, x, y):
        secret = 0
        for i in range(1, threshold + 1):
            share = images[i - 1][x, y]
            num, den = 1, 1
            for j in range(1, threshold + 1):
                if i != j:
                    num *= -j
                    den *= (i - j)
            lagrange = share * num / den
            secret += lagrange
        secret %= 256
        return int(secret)

    # 逐位置读取每张图片的灰度值并进行处理
    for y in range(height):
        for x in range(width):
            new_pixels = reconstruct_secret(images, x, y)
            new_image.putpixel((x, y), new_pixels)
            
    new_image.save("/Users/colofuluangang/Desktop/code/Shamir/G通道恢复秘密图片/G恢复.png")

In [29]:
from PIL import Image

if verification_failed == False:
    new_image = Image.new("L", (width, height))
    num_shares = 6  # 要创建的份额总数
    threshold = 4  # 重构秘密所需的最小份额数量

    # 准备用于存储所有子图片数据的列表
    images = []
    for i in range(threshold):
        file_path = f"/Users/colofuluangang/Desktop/code/Shamir/B通道子秘密图片/B{i}.png"
        img = Image.open(file_path)
        images.append(img.load())  # 将像素数据加载到内存，避免多次读取文件

    # Shamir的秘密共享方案 - 优化的秘密重构函数
    def reconstruct_secret(images, x, y):
        secret = 0
        for i in range(1, threshold + 1):
            share = images[i - 1][x, y]
            num, den = 1, 1
            for j in range(1, threshold + 1):
                if i != j:
                    num *= -j
                    den *= (i - j)
            lagrange = share * num / den
            secret += lagrange
        secret %= 256
        return int(secret)

    # 逐位置读取每张图片的灰度值并进行处理
    for y in range(height):
        for x in range(width):
            new_pixels = reconstruct_secret(images, x, y)
            new_image.putpixel((x, y), new_pixels)
            
    new_image.save("/Users/colofuluangang/Desktop/code/Shamir/B通道恢复秘密图片/B恢复.png")

9. 将恢复的R、G、B通道图像合成一张彩色图片

In [30]:
from PIL import Image

if verification_failed == False:
    # 打开三张通道的图像
    r_channel = Image.open("/Users/colofuluangang/Desktop/code/Shamir/R通道恢复秘密图片/R恢复.png")
    g_channel = Image.open("/Users/colofuluangang/Desktop/code/Shamir/G通道恢复秘密图片/G恢复.png")
    b_channel = Image.open("/Users/colofuluangang/Desktop/code/Shamir/B通道恢复秘密图片/B恢复.png")

    # 合并图像
    merged_image = Image.merge("RGB", (r_channel, g_channel, b_channel))

    # 保存合并后的彩色图像
    merged_image.save("/Users/colofuluangang/Desktop/code/Shamir/恢复图像/恢复图像.png")

10. 如果秘密分发者在场，可以直接对比两张图片的像素值来对比图像是否被修改

In [31]:
from PIL import Image

if verification_failed == False:
    # 打开第一张图片
    image1 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/原始图片/1.png")

    # 打开第二张图片
    image2 = Image.open("/Users/colofuluangang/Desktop/code/Shamir/恢复图像/恢复图像.png")

    # 比较两张图片的像素数据
    if list(image1.getdata()) == list(image2.getdata()):
        print("两张图片完全相同")
    else:
        print("两张图片不同")

两张图片完全相同


11. 大多数场景，秘密分发者不在场，需要验证恢复的图像的签名是否正确

        注意：其中pkcs1_15.new(public_key).verify(sha256_hash, signature) 用来验证数字签名。使用 Crypto.Signature.pkcs1_15 模块中的 PKCS#1 v1.5 签名方案来验证数字签名的有效性。

        1. public_key 是用于验证数字签名的公钥。
        2. sha256_hash 是之前计算的图像数据的SHA-256哈希值。
        3. signature 是数字签名本身，您希望验证其有效性。
        4. 将公钥、哈希值和数字签名传递给 pkcs1_15.new(public_key).verify() 函数，该函数会验证数字签名是否与给定的哈希值相匹配。如果数字签名有效，那么函数将返回而不引发异常，否则将引发 ValueError 或 TypeError 异常，指示签名无效。

In [32]:
from PIL import Image
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15

if verification_failed == False:
    # 打开数字签名文件
    with open("/Users/colofuluangang/Desktop/code/Shamir/签名文件/signature.sig", "rb") as signature_file:
        signature = signature_file.read()

    # 提取图像数据的像素数据
    image = Image.open("/Users/colofuluangang/Desktop/code/Shamir/恢复图像/恢复图像.png")
    pixel_data = list(image.getdata())

    # 计算图像数据的SHA-256哈希值
    sha256_hash = SHA256.new()
    for pixel in pixel_data:
        sha256_hash.update(bytes(pixel))

    # 打开公钥文件
    with open("/Users/colofuluangang/Desktop/code/Shamir/RSA密钥对文件/public_key.pem", "rb") as public_key_file:
        public_key_data = public_key_file.read()

    # 创建RSA公钥对象
    public_key = RSA.import_key(public_key_data)

    try:
        # 使用公钥验证签名
        pkcs1_15.new(public_key).verify(sha256_hash, signature)
        print("数字签名验证成功，签名有效。")
    except (ValueError, TypeError):
        print("数字签名验证失败，签名无效。")

数字签名验证成功，签名有效。
